In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk.data
from collections import Counter
import time
import pickle
import timeit
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from pymongo import MongoClient, errors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string

Series([], Name: url, dtype: int64)

In [10]:
url.values

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]])

In [5]:
df1.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [2]:
ls data/

all-the-news/


In [3]:
df1 = pd.read_csv('data/all-the-news/articles1.csv')

In [18]:
df2 = pd.read_csv('data/all-the-news/articles2.csv')

In [22]:
df3 = pd.read_csv('data/all-the-news/articles3.csv')

In [23]:
df1pub = df1['publication']
df2pub = df2['publication']
df3pub = df3['publication']

In [5]:
df1pub.value_counts()

Breitbart           23781
CNN                 11488
New York Times       7803
Business Insider     6757
Atlantic              171
Name: publication, dtype: int64

In [24]:
df2pub.value_counts()

New York Post          17493
Atlantic                7008
National Review         6203
Talking Points Memo     5214
Guardian                4873
Buzzfeed News           4854
Fox News                4354
Name: publication, dtype: int64

In [7]:
df3pub.value_counts()

NPR                11992
Washington Post    11114
Reuters            10710
Vox                 4947
Guardian            3808
Name: publication, dtype: int64

In [8]:
Vox = 4947
#Buzzfeed_News = 854
the_atlantic = 7171
CNN = 11488
BuzzFeedNEWS = 4854


#Fox_news = 4354
National_Review = 6203
Breitbart = 23781
New_York_Post = 17493


In [9]:
Vox + the_atlantic +CNN#+BuzzFeedNEWS #Buzzfeed_News + 

23606

In [10]:
National_Review +Breitbart +New_York_Post #Fox_news + 

47477

In [16]:
Breitbart_Articles['content'][7803]

'On Tuesday’s broadcast of CNN’s “Situation Room,” CNN Senior Washington Correspondent Jeff Zeleny stated that Chelsea Manning’s transition from a man to a woman “certainly played into” President Obama’s decision to commute Manning’s sentence, and “Without that, it’s hard to imagine, I think, this president would have done that. ”  Zeleny said, “I think a question that this president the White House will have to answer here  —   will answer, and I think it’s an important one, if  —   how much was the personal story of Chelsea Manning involved in this, because the outcry from the left was so strong on this. And she’s having a difficult time in federal prison, no question. But, to me, that is a central question here. Without that, you have to wonder if the outcome would be the same. I think it might not be. ” He added, “[B]ecause she transitioned from a man to a woman, I think all of that certainly played into this. Without that, it’s hard to imagine, I think, this president would have d

In [7]:
Breitbart_Articles['content'].head()

7803    On Tuesday’s broadcast of CNN’s “Situation Roo...
7804    A group of American spring break revelers repo...
7805      “honour crimes” have risen by 40 per cent in...
7806    Former Breitbart Senior Editor MILO has announ...
7807    The focus of the continuous media reports of a...
Name: content, dtype: object

In [4]:
Breitbart_Articles = (df1[df1["publication"]=="Breitbart"])
National_Review_Articles = (df2[df2["publication"]=="National Review"])
New_York_Post_Articles = (df2[df2["publication"]=="New York Post"])


NameError: name 'df2' is not defined

In [14]:
Vox_Articles = (df3[df3["publication"]=="Vox"])
Atlantic_Articles1 = (df1[df1["publication"]=="Atlantic"])
Atlantic_Articles = Atlantic_Articles1.append((df2[df2["publication"]=="Atlantic"]))
CNN_Articles = (df1[df1["publication"]=="CNN"])
#Buzzfeed_Articles = (df2[df2["publication"]=="Buzzfeed News"])



In [25]:
Buzzfeed_Articles = (df2[df2["publication"]=="Buzzfeed News"])
Fox_Articles = (df2[df2["publication"]=="Fox News"])

In [15]:
Conservative_Pubs = Breitbart_Articles
Conservative_Pubs = Conservative_Pubs.append(National_Review_Articles)
Conservative_Pubs = Conservative_Pubs.append(New_York_Post_Articles)

Liberal_Pubs = Vox_Articles
Liberal_Pubs = Liberal_Pubs.append(Atlantic_Articles)
Liberal_Pubs = Liberal_Pubs.append(CNN_Articles)

x=1

In [16]:
Liberal_Pubs.shape

(23614, 10)

In [17]:
Conservative_Pubs.shape

(47477, 10)

In [18]:
join_eval = Conservative_Pubs['publication']

In [19]:
join_eval.value_counts()

Breitbart          23781
New York Post      17493
National Review     6203
Name: publication, dtype: int64

In [55]:
con_text = Conservative_Pubs['content']
lib_text = Liberal_Pubs['content']

In [24]:
Conservative_Pubs.columns

Index(['Unnamed: 0', 'id', 'title', 'publication', 'author', 'date', 'year',
       'month', 'url', 'content'],
      dtype='object')

In [25]:
len(lib_text)

23614

In [56]:
lib_words = {}
for i in range(0,len(lib_text)):
    if i%100 ==0:
        print("done ",i," out of ",len(lib_text))
    lib_text[i] = str.split(lib_text[i])
for i in range(0,len(con_text)):
    if i%100 ==0:
        print("done ",i," out of ",len(lib_text))
    con_text[i] = str.split(con_text[i])

for doc in lib_text.head():
    for word in doc:
        if word in lib_words:
            lib_words[word] = lib_words[word] +1
        else:
            lib_words[word] = 1

            
#for doc in con_text.head():
#    for word in doc:
#        if word in con_words:
 #           con_words[word] = con_words[word] +1
 #       else:
  #          con_words[word] = 1

done  0  out of  23614


TypeError: descriptor 'split' requires a 'str' object but received a 'list'

In [ ]:
# tokenize
docs = [word_tokenize(str(content)) for content in con_text]

# Remove stopwords
stop = set(stopwords.words('english'))
docs = [[word for word in words if ((word not in stop)and (word not in string.punctuation))] for words in docs]

# lemmatize
wordnet = WordNetLemmatizer()
docs_wordnet = [[wordnet.lemmatize(word) for word in words] for words in docs]
words_rucksack = []
global_con_cnt = Counter()
for doc in docs_wordnet:
    local_con_list = []
    for word in doc:
        if word not in local_con_list:
            local_con_list.append(word)
    local_count = Counter(local_con_list)
    global_con_cnt = global_con_cnt + local_count
    
    #for word in doc:
        #words_rucksack.append(word)
con_cnt = Counter(words_rucksack)

In [153]:
c = Counter(a=3, b=1)
d = Counter(a=1, b=2, c=1)
c + d

Counter({'a': 4, 'b': 3, 'c': 1})

In [96]:
# tokenize
docs = [word_tokenize(str(content)) for content in lib_text]

# Remove stopwords
stop = set(stopwords.words('english'))
docs = [[word for word in words if ((word not in stop)and (word not in string.punctuation))] for words in docs]

# lemmatize
wordnet = WordNetLemmatizer()
docs_wordnet = [[wordnet.lemmatize(word) for word in words] for words in docs]
words_rucksack = []
for doc in docs_wordnet:
    for word in doc:
        words_rucksack.append(word)
cnt = Counter(words_rucksack)
#words_rucksack = [word for word in doc for doc in docs_wordnet]
#docs_wordnet = [(wordnet.lemmatize(word) for word in words) for words in docs]
#for doc in docs_wordnet:
    #for word in doc:
#    if word in lib_words:
#        lib_words[word] = lib_words[word] +1
#    else:
#        lib_words[word] = 1

In [95]:
i = 0
for content in lib_text:
    i=i+1
    word_tokenize(str(content))
    if i == 5000:
        time.sleep(1)


In [158]:
len(con_text)
pickle.dump(len(con_text),open('con_doc_count.p','wb'))
pickle.dump(len(lib_text),open('lib_doc_count.p','wb'))
    

In [102]:
pickle.dump(con_cnt,open('con_count.p','wb'))


In [156]:
len(lib_text)

23614

In [205]:
lib_words = 0
for doc in lib_text:
    lib_words = lib_words + len(word_tokenize(str(doc)))

In [207]:
con_words = 0
for doc in con_text:
    con_words = con_words + len(word_tokenize(str(doc)))

In [212]:
con_words

30006946

In [ ]:
con_text = pd.DataFrame(con_text).reset_index(drop=True)
con_text.drop('index', axis=1)

In [249]:
con_text = con_text['content']

In [251]:
#con_text

In [103]:
con_c = pickle.load(open('con_count.p','rb'))
lib_c = pickle.load(open('lib_count.p','rb'))

In [46]:


C_lib_test_articles = 0
C_con_test_articles = 0
C_zero_articles = 0

C_lib_test_articles_strength = 0
C_con_test_articles_strength = 0

i = 0
for article in Fox_Articles['content']:
    if i < 6000:
        l,c = classify(article)
        if (l>c):
            C_lib_test_articles = C_lib_test_articles +1
            C_lib_test_articles_strength = C_lib_test_articles_strength +1 * l
        elif(c>l):
            C_con_test_articles = C_con_test_articles +1
            C_con_test_articles_strength = C_con_test_articles_strength +1 * c
        else:
            C_zero_articles = C_zero_articles + 1
        i = i + 1    


In [40]:
C_lib_test_articles

75

In [41]:
C_con_test_articles

923

In [52]:
Fox_Articles['content'].shape

(4354,)

In [47]:
L_lib_test_articles = 0
L_con_test_articles = 0
L_zero_articles = 0

L_lib_test_articles_strength = 0
L_con_test_articles_strength = 0

i = 0
for article in Buzzfeed_Articles['content']:
    if i < 6000:
        l,c = classify(article)
        if (l>c):
            L_lib_test_articles = L_lib_test_articles +1
            L_lib_test_articles_strength = L_lib_test_articles_strength +1 * l
        elif(c>l):
            L_con_test_articles = L_con_test_articles +1
            L_con_test_articles_strength = L_con_test_articles_strength +1 *c
        else:
            L_zero_articles = L_zero_articles + 1
        i = i + 1  

In [33]:
Buzzfeed_Articles.shape

(4854, 10)

In [49]:
print('L_lib_test_articles: ',L_lib_test_articles)
print('L_lib_test_articles_strength: ',L_lib_test_articles_strength)
print('L_con_test_articles: ',L_con_test_articles)
print('L_con_test_articles_strength: ',L_con_test_articles_strength)
print('L_zero_articles: ',L_zero_articles)

L_lib_test_articles:  142
L_lib_test_articles_strength:  2.826863900798512e-06
L_con_test_articles:  825
L_con_test_articles_strength:  0.2636346231712514
L_zero_articles:  33


In [50]:
print('C_lib_test_articles: ',C_lib_test_articles)
print('C_lib_test_articles_strength: ',C_lib_test_articles_strength)
print('C_con_test_articles: ',C_con_test_articles)
print('C_con_test_articles_strength: ',C_con_test_articles_strength)
print('C_zero_articles: ',C_zero_articles)

C_lib_test_articles:  75
C_lib_test_articles_strength:  1.066738907674856e-07
C_con_test_articles:  923
C_con_test_articles_strength:  0.31382835999574804
C_zero_articles:  2


In [44]:
L_con_test_articles

825

In [45]:
L_zero_articles

33

In [27]:
def classify(text):
    text = str(text)
    con_c = pickle.load(open('con_count.p','rb'))
    lib_c = pickle.load(open('lib_count.p','rb'))
    #con_c = pickle.load(open('con_doc_count.p','rb'))
    #lib_c = pickle.load(open('lib_doc_count.p','rb'))
    array_words = text.split()
    lib_strength = 1
    con_strength = 1
########### tokenize
    token_text = word_tokenize(str(text))

# Remove stopwords
    stop = set(stopwords.words('english'))
    doc = [word for word in token_text if ((word not in stop)and (word not in string.punctuation))]

# lemmatize
    wordnet = WordNetLemmatizer()
    docs_wordnet = [wordnet.lemmatize(word) for word in doc]
    #print(str(docs_wordnet))
    words_rucksack = []
############
    for word in docs_wordnet:
        #print('word: ',word)
        if word in con_c:
            con_tf = con_c[word]
        else:
            con_tf = 1
        if word in lib_c:
            lib_tf = lib_c[word]
        else:
            lib_tf = 1
        #print('--------------------------')
        #print('term: ',word)
        #print('con count: ',con_tf)
        #print('lib count: ',lib_tf)    
        #print('prior con s: ',con_strength) 
        #print('prior lib s: ',lib_strength) 
        if (con_tf > 1) or (lib_tf > 1):
            lib_strength = lib_strength * np.sqrt(lib_tf/(lib_tf+con_tf))#devide by length of lib corpus
            con_strength = con_strength * np.sqrt(con_tf/(lib_tf+con_tf))
        #print('post con s: ',con_strength) 
        #print('post lib s: ',lib_strength)
    return lib_strength, con_strength

In [198]:
#l,c = classify(lib_text[26511])

In [192]:
lib_text.head()

26510     People have dreamed about flying cars for dec...
26511     US intelligence agencies believe that this ye...
26512     Minnesota Rep. Keith Ellison hadn’t been camp...
26513     One of the biggest and most important science...
26514     I gave up drinking when my pancreas exploded....
Name: content, dtype: object

In [162]:
def order_cost(pred,expected,value):
    return abs(pred[value]-expected[value])
    
def evaluation(pred,expected):
    error = 0
    for outlet in pred:
        error = error + order_cost(pred,expected,outlet)
    
#def test(pred,expected,value):
    
        
        
        #scheme 1: classification left or right
        #scheme 2: order
        #scheme 3: how close to target 
    

In [199]:
c

1.0060765049821886e-121

In [200]:
l

4.243571400758148e-118

In [160]:
#l,c = classify(test_article)

In [129]:
l

2340.1812895233998

In [130]:
c

2429.8187104766025

In [136]:
df1['content'].head()

0    WASHINGTON  —   Congressional Republicans have...
1    After the bullet shells get counted, the blood...
2    When Walt Disney’s “Bambi” opened in 1942, cri...
3    Death may be the great equalizer, but it isn’t...
4    SEOUL, South Korea  —   North Korea’s leader, ...
Name: content, dtype: object

In [161]:
#lib_c

In [ ]:
# use all docs vs use strong docs with weight vs 